In [8]:
from keras.models import load_model
import cv2
import numpy as np
from keras.preprocessing.image import img_to_array

In [9]:
model=load_model('emotion.h5')
face_classifier = cv2.CascadeClassifier('/home/user/.local/lib/python3.8/site-packages/cv2/data/haarcascade_frontalface_default.xml')
class_labels={0: 'Angry', 1: 'Happy', 2: 'Neutral', 3: 'Sad', 4: 'Surprise'}
def face_detector(img):
    gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces=face_classifier.detectMultiScale(gray,1.3,5)
    if len(faces) == 0:
        return (0,0,0,0), np.zeros((48,48), np.uint8), img
    else:
        for (x,y,w,h) in faces:
            cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
            roi_gray = gray[x:x+w,y:y+h]
            roi_gray = cv2.resize(roi_gray, (48, 48), interpolation = cv2.INTER_AREA)
            roi_color=img[x:x+w,y:y+h]
            return (x,w,y,h), roi_gray, img
cap=cv2.VideoCapture(0)
while True:
    ret,frame=cap.read()
    if ret==True:
        rect,face,image=face_detector(frame)
        if np.sum([face]) != 0.0:
            roi = face.astype("float") / 255.0
            roi = img_to_array(roi)
            roi = np.expand_dims(roi, axis=0)
            preds = model.predict(roi)[0]
            label = class_labels[preds.argmax()] 
            label_position = (rect[0] + int((rect[1]/2)), rect[2] + 25)
            cv2.putText(image, label, label_position , cv2.FONT_HERSHEY_SIMPLEX,2, (0,255,0), 3)
        else:
            cv2.putText(image, "No Face Found", (20, 60) , cv2.FONT_HERSHEY_SIMPLEX,2, (0,255,0), 3)
        cv2.imshow('image',image)
        if cv2.waitKey(1)==13:
            break
cap.release()
cv2.destroyAllWindows()